# Read Dataset

In [14]:
# generate 6000 samples
import numpy as np
import os
import sys
sys.path.append('../..')

import math
from fedrpdp.accountants.rdp import RDPAccountant
from fedrpdp.record_budget_generation import EPSILONS_GEN_FUNC
from fedrpdp.autodp.optimizers.pdp_optimizer import make_optimizer_class as pdp_optim_cls


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Subset, TensorDataset
from torchvision.datasets import MNIST, CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor

In [15]:
# def train(model, device, train_loader, optimizer, epoch, running_norms, disable_dp=False, delta=1e-5):
#     model.train()
#     criterion = nn.CrossEntropyLoss()
#     losses = []

#     data, target = next(iter(train_loader))
#     correct = 0
#     data, target = data.to(device), target.to(device)
#     optimizer.zero_grad()
#     output = model(data)

#     # compute train acc
#     pred = output.argmax(
#             dim=1, keepdim=True
#         )  # get the index of the max log-probability
#     correct = pred.eq(target.view_as(pred)).sum().item() 

#     loss = criterion(output, target)
#     loss.backward()
#     # gradient_norms is a Tensor with size being equal to `dataset_size`
#     gradient_norms = optimizer.step(running_norms)
#     gradient_norms_sq = gradient_norms * gradient_norms
#     losses.append(loss.item())

#     # for data, target in train_loader:
#     #     correct = 0
#     #     data, target = data.to(device), target.to(device)
#     #     optimizer.zero_grad()
#     #     output = model(data)

#     #     # compute train acc
#     #     pred = output.argmax(
#     #             dim=1, keepdim=True
#     #         )  # get the index of the max log-probability
#     #     correct = pred.eq(target.view_as(pred)).sum().item() 

#     #     loss = criterion(output, target)
#     #     loss.backward()
#     #     gradient_norms = optimizer.step(running_norms)
#     #     gradient_norms_sq = gradient_norms * gradient_norms
#     #     losses.append(loss.item())

#     if not disable_dp:
#         # Note that we only show the 1st point's cumulative privacy cost.
#         epsilon_1, best_alpha_1 = optimizer.privacy_engine.get_privacy_spent(0, delta)
#         epsilon_2, best_alpha_2 = optimizer.privacy_engine.get_privacy_spent(40000, delta)
#         epsilon_3, best_alpha_3 = optimizer.privacy_engine.get_privacy_spent(50000, delta)
#         print(
#             f"Train Epoch: {epoch} \t"
#             f"Loss: {np.mean(losses):.6f} "
#             f"Acc: {correct/60000.0:.6f} "
#             f"δ: {delta} "
#             f"ε1 = {epsilon_1:.2f} for α1 = {best_alpha_1}, "
#             f"ε2 = {epsilon_2:.2f} for α2 = {best_alpha_2}, "
#             f"ε3 = {epsilon_3:.2f} for α3 = {best_alpha_3}. "
#         )
#     else:
#         print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f}")

#     return gradient_norms_sq


# def test(model, device, test_loader):
#     model.eval()
#     criterion = nn.CrossEntropyLoss()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += criterion(output, target).item()  # sum up batch loss
#             pred = output.argmax(
#                 dim=1, keepdim=True
#             )  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print(
#         "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
#             test_loss,
#             correct,
#             len(test_loader.dataset),
#             100.0 * correct / len(test_loader.dataset),
#         )
#     )
#     return correct / len(test_loader.dataset)

# Experiments on MNIST dataset

In [16]:


# project_abspath = os.path.dirname(os.getcwd())
# print(project_abspath)

# DATASET_NAME = 'mnist' # or 'cifar10'
# DATA_ROOT = '/data/privacyGroup/liujunxu/datasets/{}'.format(DATASET_NAME)
# RES_ROOT = project_abspath + '/results/sgd/{}'.format(DATASET_NAME)
# if not os.path.exists(RES_ROOT):
#     os.makedirs(RES_ROOT)
    
# IMAGENET_MEAN = {'mnist':[0.5], 'cifar10':[0.485, 0.456, 0.406]}
# IMAGENET_STD = {'mnist':[0.5], 'cifar10':[0.229, 0.224, 0.225]}
# if DATASET_NAME == 'mnist':
#     train_data = MNIST(DATA_ROOT,
#                     train=True,
#                     download=True,
#                     transform=Compose([ToTensor(), Normalize(IMAGENET_MEAN[DATASET_NAME], IMAGENET_STD[DATASET_NAME])]))
#     test_data = MNIST(DATA_ROOT, 
#                   train=False, 
#                   download=True, 
#                   transform=Compose([ToTensor(), Normalize(IMAGENET_MEAN[DATASET_NAME], IMAGENET_STD[DATASET_NAME])]))

# elif DATASET_NAME == 'cifar10':
#     train_data = CIFAR10(DATA_ROOT, 
#                     train=True, 
#                     download=True, 
#                     transform=Compose([ToTensor(), Normalize(IMAGENET_MEAN[DATASET_NAME], IMAGENET_STD[DATASET_NAME])]))
#     test_data = CIFAR10(DATA_ROOT, 
#                   train=False, 
#                   download=True, 
#                   transform=Compose([ToTensor(), Normalize(IMAGENET_MEAN[DATASET_NAME], IMAGENET_STD[DATASET_NAME])]))
    
# kwargs = {"num_workers": 1, "pin_memory": True}
# device = 'cuda:2'

# train_loader = DataLoader(
#     train_data,
#     batch_size=60000, # use all data points
#     shuffle=False,
#     **kwargs,
# )
# test_loader = DataLoader(
#     test_data,
#     batch_size=1024,
#     shuffle=True,
#     **kwargs,
# )

In [17]:
# class SampleConvNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
#         self.conv2 = nn.Conv2d(16, 32, 4, 2)
#         self.fc1 = nn.Linear(32 * 4 * 4, 32)
#         self.fc2 = nn.Linear(32, 10)

#     def forward(self, x):
#         # x of shape [B, 1, 28, 28]
#         x = F.relu(self.conv1(x))   # -> [B, 16, 14, 14]
#         x = F.max_pool2d(x, 2, 1)   # -> [B, 16, 13, 13]
#         x = F.relu(self.conv2(x))   # -> [B, 32, 5, 5]
#         x = F.max_pool2d(x, 2, 1)   # -> [B, 32, 4, 4]
#         x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
#         x = F.relu(self.fc1(x))     # -> [B, 32]
#         x = self.fc2(x)             # -> [B, 10]
#         return x

#     def name(self):
#         return "SampleConvNet"

## 1) Uniform privacy budgets settings

In [18]:
# from torchdp import PrivacyEngine

# epochs = 150
# budget = 11200
# lr = .2
# sigma = 170
# max_per_sample_grad_norm = 10
# delta = 1e-5
# disable_dp = False
    
# run_results = []
# active_points = []
# running_gradient_sq_norms = [0]

# model = SampleConvNet().to(device)

# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0)
# if not disable_dp:
#     privacy_engine = PrivacyEngine(
#         model,
#         batch_size=60000,
#         sample_size=len(train_loader.dataset),
#         alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 80)),
#         noise_multiplier=sigma,
#         max_grad_norm=max_per_sample_grad_norm,
#         norm_sq_budget = budget,
#         should_clip = True,
#     )
#     privacy_engine.attach(optimizer)

# for epoch in range(1, epochs + 1):
#     gradient_norms = train(model, device, train_loader, optimizer, epoch, running_gradient_sq_norms[-1])

#     # update running squared grad norms
#     running_gradient_sq_norms.append(running_gradient_sq_norms[-1] + gradient_norms)

#     # add new test accuracy 
#     run_results.append(test(model, device, test_loader))
#     active_points.append(torch.sum(running_gradient_sq_norms[-1] < budget).cpu().numpy())
#     if torch.sum(running_gradient_sq_norms[-1] - budget) == 0:
#         break
	   
# alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64))
# eps_val = min([alpha/2*budget/(sigma**2 * max_per_sample_grad_norm**2) + np.log(1/delta)/(alpha-1) for alpha in alphas])

In [19]:
# import matplotlib.pyplot as plt
# path1 = "accuracies.pdf"
# path2 = "active_points.pdf"

# plt.plot(active_points)
# plt.axvline(int(budget/(max_per_sample_grad_norm**2)), 0, 1, c='m', linewidth = 3)
# plt.ylim(0,65000)
# plt.title(f"ε = {eps_val:.1f}, δ =1e-5", fontsize = 22)
# plt.ylabel(f"Number of active points", fontsize = 22)
# plt.xlabel("Step", fontsize = 22)
# plt.savefig(path2, bbox_inches='tight')

In [20]:
# plt.clf()
    
# plt.plot(range(1,len(run_results) + 1), run_results, linewidth = 3)
# plt.axvline(int(budget/(max_per_sample_grad_norm**2)), 0, 1, c='m', linewidth = 3)
# plt.xlabel("Step", fontsize = 22)
# plt.ylabel("Test accuracy", fontsize = 22)
# plt.title(f"ε = {eps_val:.1f}, δ =1e-5", fontsize = 22)
# plt.yticks([0.8, 0.825, 0.85, 0.875, 0.9, 0.925, 0.95, 0.975, 1.0])
# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)
# plt.ylim(0.875,0.975)
# plt.xlim(0,epochs)
# plt.grid(True)
# plt.savefig(path1, bbox_inches='tight')

## 2) Varied privacy budgets settings 

In [21]:
# epochs = 150
# budgets = np.array([11200]*40000 + [14400]*10000 + [19800]*10000)
# lr = .2
# sigma = 170
# max_per_sample_grad_norm = 10
# delta = 1e-5

In [22]:
# from torchdp import PrivacyEngine

# disable_dp = False

# run_results = []
# active_points = []
# running_gradient_sq_norms = [0]

# model = SampleConvNet().to(device)

# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0)
# if not disable_dp:
#     privacy_engine = PrivacyEngine(
#         model,
#         batch_size=60000,
#         sample_size=len(train_loader.dataset),
#         alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 80)),
#         noise_multiplier=sigma,
#         max_grad_norm=max_per_sample_grad_norm,
#         norm_sq_budget = budgets,
#         should_clip = True,
#     )
#     privacy_engine.attach(optimizer)

# for epoch in range(1, epochs + 1):
#     gradient_norms = train(model, device, train_loader, optimizer, epoch, running_gradient_sq_norms[-1])

#     # update running squared grad norms
#     running_gradient_sq_norms.append(running_gradient_sq_norms[-1] + gradient_norms)

#     # add new test accuracy 
#     run_results.append(test(model, device, test_loader))
#     num_active_points = np.sum(running_gradient_sq_norms[-1].cpu().numpy() < np.array(budgets))
#     active_points.append(num_active_points)
#     if num_active_points == 0:
#         break
	   
# alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64))
# eps_val = min([alpha/2*budgets/(sigma**2 * max_per_sample_grad_norm**2) + np.log(1/delta)/(alpha-1) for alpha in alphas])

# Experiments on Medical dataset

## Fliter

In [23]:
from fedrpdp.datasets.fed_heart_disease import (
    Baseline,
    BaselineLoss,
    FedHeartDisease,
    metric,
)
from fedrpdp.utils.flamby_utils import evaluate_model_on_tests

metrics_dict = {"AUC": metric}
use_gpu = True
debug = False
disable_dp = False

budgets = np.array([0.15]*400 + [0.3]*50 + [0.45]*36)
sigma = 100
max_per_sample_grad_norm = 1.1
delta = 1e-5

num_workers_torch = 20
device = 'cuda:0'
NUM_EPOCHS_POOLED = 100
LR = 0.05

train_data = FedHeartDisease(train=True, pooled=True, debug=debug)
test_data = FedHeartDisease(train=False, pooled=True, debug=debug)
train_loader = DataLoader(
    train_data,
    num_workers=num_workers_torch,
    batch_size=len(train_data),
    shuffle=True,
)
test_loader = DataLoader(
    test_data,
    num_workers=num_workers_torch,
    batch_size=len(test_data),
    shuffle=False,
)

## rPDP-SGD 

In [24]:
from fedrpdp.autodp.analysis.privcost_utils import PrivCostEstimator
MIN_BUDGET, MAX_BUDGET, VALID_BUDGET = 0.1, 10.0, 1.0
gen_bounded_priv_bgts_func = lambda priv_bgts: np.array([min(max(x, MIN_BUDGET), MAX_BUDGET) for x in priv_bgts])

privCostCurve = PrivCostEstimator(
    sigma = sigma, 
    inner_iters = 1, 
    outer_iters = 0, 
    outer_ratio = 1.0,
    delta = delta).estimator(est_object="probs")

sampling_probs = np.array([privCostCurve(eps) for eps in budgets])

maximum budget: 0.04804357848189008 minimum budget: 0.012406216138705024
[0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01  0.011 0.012
 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023 0.024
 0.025 0.026 0.027 0.028 0.029 0.03  0.031 0.032 0.033 0.034 0.035 0.036
 0.037 0.038 0.039 0.04  0.041 0.042 0.043 0.044 0.045 0.046 0.047 0.048
 0.049 0.05  0.051 0.052 0.053 0.054 0.055 0.056 0.057 0.058 0.059 0.06
 0.061 0.062 0.063 0.064 0.065 0.066 0.067 0.068 0.069 0.07  0.071 0.072
 0.073 0.074 0.075 0.076 0.077 0.078 0.079 0.08  0.081 0.082 0.083 0.084
 0.085 0.086 0.087 0.088 0.089 0.09  0.091 0.092 0.093 0.094 0.095 0.096
 0.097 0.098 0.099 0.1   0.11  0.12  0.13  0.14  0.15  0.16  0.17  0.18
 0.19  0.2   0.21  0.22  0.23  0.24  0.25  0.26  0.27  0.28  0.29  0.3
 0.31  0.32  0.33  0.34  0.35  0.36  0.37  0.38  0.39  0.4   0.41  0.42
 0.43  0.44  0.45  0.46  0.47  0.48  0.49  0.5   0.51  0.52  0.53  0.54
 0.55  0.56  0.57  0.58  0.59  0.6   0.61  0.62  0.63  0.

/home/wamdm/miniconda3/envs/liujunxu_tf1/lib/python3.7/site-packages/autodp/rdp_acct.py:724: RuntimeWarning: divide by zero encountered in log
  + (alpha+1-jvec[0:alpha-1])*np.log(1-prob))
/home/wamdm/miniconda3/envs/liujunxu_tf1/lib/python3.7/site-packages/autodp/utils.py:8: RuntimeWarning: invalid value encountered in subtract
  return a+np.log(np.sum(np.exp(x-a)))
/home/wamdm/miniconda3/envs/liujunxu_tf1/lib/python3.7/site-packages/autodp/rdp_acct.py:725: RuntimeWarning: divide by zero encountered in log
  results[alpha-1] = utils.stable_logsumexp_two((alpha-1)*np.log(1-prob)


In [25]:
class NonUniformBatchSampler:
    def __init__(self, dataset, probs):
        self.length = len(dataset)
        self.probs = probs
       
    def __iter__(self):      
        indices = np.where(np.random.random(self.length) <= self.probs)[0]
        yield indices

    def __len__(self):
        return self.iterations

def dp_train(model, device, train_loader, optimizer, epoch, disable_dp=False, delta=1e-5):
    model.train()
    criterion = BaselineLoss()
    losses = []

    train_loader = DataLoader(
        train_data, 
        batch_sampler=NonUniformBatchSampler(train_data, sampling_probs),
        shuffle=False
    )
    data, target = next(iter(train_loader))
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)

    k, v = next(iter(metrics_dict.items()))
    train_metric = v(
        target.detach().cpu().numpy(), output.detach().cpu().numpy()
    )
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if not disable_dp:
        # Note that we only show the 1st point's cumulative privacy cost.
        epsilon_1, best_alpha_1 = optimizer.privacy_engine.get_privacy_spent(0, delta)
        epsilon_2, best_alpha_2 = optimizer.privacy_engine.get_privacy_spent(400, delta)
        epsilon_3, best_alpha_3 = optimizer.privacy_engine.get_privacy_spent(450, delta)
        print(
            f"Train Epoch: {epoch} \t"
            f"Loss: {np.mean(losses):.6f} "
            f"{k}: {train_metric:.6f} "
            f"δ: {delta} "
            f"ε1 = {epsilon_1:.2f} for α1 = {best_alpha_1}, "
            f"ε2 = {epsilon_2:.2f} for α2 = {best_alpha_2}, "
            f"ε3 = {epsilon_3:.2f} for α3 = {best_alpha_3}. "
        )
    else:
        print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f}")


def test(model, device, test_loader):
    model.eval()
    criterion = BaselineLoss()

    with torch.no_grad():
        data, target = next(iter(test_loader))
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss = criterion(output, target).item()
        
        for k, v in metrics_dict.items():
            test_metric = v(
                target.detach().cpu().numpy(), output.detach().cpu().numpy()
            )
        print(
            "\nTest set: Average loss: {:.4f}, {}: {:.2f}\n".format(
                test_loss,
                k,
                test_metric,
            )
        )

    return test_metric

In [26]:
from torchdp import PrivacyEngine

run_results = []
model = Baseline().to(device)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0)

def generate_rdp_orders():
    dense = 1.07
    alpha_list = [int(dense ** i + 1) for i in range(int(math.floor(math.log(1000, dense))) + 1)]
    alpha_list = np.unique(alpha_list)
    return alpha_list

privacy_engine = PrivacyEngine(
    model,
    batch_size=len(train_loader.dataset),
    sample_size=len(train_loader.dataset),
    # alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 80)),
    alphas = generate_rdp_orders(),
    noise_multiplier=sigma,
    max_grad_norm=max_per_sample_grad_norm,
    norm_sq_budget = budgets,
    should_clip = True,
)
privacy_engine.attach(optimizer)

# pdp_optim = pdp_optim_cls(torch.optim.SGD)
# optimizer = pdp_optim(
#     l2_norm_clip=max_per_sample_grad_norm,
#     gauss_std=sigma * max_per_sample_grad_norm,
#     minibatch_size=1,
#     microbatch_size=len(train_data),
#     params=model.parameters(),
#     lr=LR)

for epoch in range(1, NUM_EPOCHS_POOLED + 1):
    dp_train(model, device, train_loader, optimizer, epoch)
    # add new test accuracy 
    run_results.append(test(model, device, test_loader))


TypeError: dp_step() missing 1 required positional argument: 'running_norms'